# Retraitement des données horaires de la qualité de l'air (format xml)

#### Nous disposons de plusieurs datasets :

- une collection de fichiers xml avec l'historique horaire de la qualité de l'air pour plusieurs région de France (1 fichier = 1 journée)
- une base de donnée B
- une base de donnée D

L'objectif est de parser les différents fichiers xml pour obtenir un tableau clair des données, pour la région qui nous intéresse (Paris)

In [20]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
#import ipdb
from datetime import datetime

In [21]:
#chemin du fichier xml qui servira de référence pour l'algo de construction du dataset
url_xml_test = r"https://files.data.gouv.fr/lcsqa/concentrations-de-polluants-atmospheriques-reglementes/old/2019/fr-2019-e2-2018-2018-12-31-23-v.xml"

# Création du tableau de vérification

In [63]:
tbl_verification = pd.DataFrame({'code verification':[1,2,3],'description':['Verified','Preliminary verified','Not verified']})
tbl_verification

,code verification,description
0,1,Verified
1,2,Preliminary verified
2,3,Not verified


# Création du tableau de validité

In [66]:
tbl_validite = pd.DataFrame({'code verification':[-99,-1,1,2,3],
                                 'description': ['Not valid due to station maintenance or calibration in the observationvalidity vocabulary',
                                                 'Not valid',
                                                 'Valid',
                                                 'Valid, but below detection limit measurement value given in the observationvalidity vocabulary',
                                                 'Valid, but below detection limit and number replaced by 0.5*detection limit in the observationvalidity vocabulary']})
tbl_validite

,code verification,description
0,-99,Not valid due to station maintenance or calibr...
1,-1,Not valid
2,1,Valid
3,2,"Valid, but below detection limit measurement v..."
4,3,"Valid, but below detection limit and number re..."


# Récupération de la liste des polluants

In [22]:
#Récupération de la liste des polluants (ils ont préalablement été transposés depuis le site dans un fichier csv)
#La liste est non exhaustive et comprend seulement les polluants qui sont mesurés dans le dataset
tbl_polluants = pd.read_csv("polluants.csv",sep=";")

tbl_polluants.head(4)

,Code,Nom,Notation
0,1,Sulphur dioxide (air),SO2
1,3,Strong acidity (air),SA
2,4,Total suspended particulates (aerosol),SPM
3,5,Particulate matter < 10 µm (aerosol),PM10


# Récupération des données sur les points de prélèvement

In [26]:
filename = "D_points_prelevement.xml"
#Attention, très lent
soup_spots = BeautifulSoup(open(filename),'xml')

Aperçu non exhaustif : 
    
```xml
  ...
  <gml:featureMember>
  ...
  </gml:featureMember>
  <gml:featureMember>
  ...
          <base:localId>SPO-FR01001_38</base:localId>
            ...
```
<code style="background:yellow;color:black">---> Code du point de prélèvement </code>
```xml
                   ...
                    <gn:GeographicalName>
                      ...
                      <gn:spelling>
                        <gn:SpellingOfName>
                          <gn:text>SCHILTIGHEIM</gn:text>
                        </gn:SpellingOfName>
                      </gn:spelling>
                    </gn:GeographicalName>
                  </ad:adminUnit>
                  <ad:locatorDesignator>5 rue de Madrid, Espace Européen de l'Entreprise</ad:locatorDesignator>
                  <ad:postCode>67300</ad:postCode>
               
```
<code style="background:yellow;color:black">---> Ville + Département </code>
```xml
   ...
  </gml:featureMember>

```

In [28]:
all_xml_data_spots = soup_spots.find_all("gml:featureMember")

In [40]:
#Aperçu d'un élément (l'élément 0 correspond aux données générales du fichier donc à ne pas prendre en compte)
all_xml_data_spots[1]

<gml:featureMember>
<aqd:AQD_SamplingPoint gml:id="SPO-FR01001_38">
<ef:inspireId>
<base:Identifier>
<base:localId>SPO-FR01001_38</base:localId>
<base:namespace>FR.LCSQA-INERIS.AQ</base:namespace>
<base:versionId>1</base:versionId>
</base:Identifier>
</ef:inspireId>
<ef:mediaMonitored xlink:href="http://inspire.ec.europa.eu/codeList/MediaValue/air"/>
<ef:responsibleParty>
<base2:RelatedParty>
<base2:individualName>
<gmd:LocalisedCharacterString>Julien PIECHOWSKI</gmd:LocalisedCharacterString>
</base2:individualName>
<base2:organisationName>
<gmd:LocalisedCharacterString>ATMO GRAND EST</gmd:LocalisedCharacterString>
</base2:organisationName>
<base2:contact>
<base2:Contact>
<base2:address>
<ad:AddressRepresentation>
<ad:adminUnit>
<gn:GeographicalName>
<gn:language>fra</gn:language>
<gn:nativeness nilReason="missing" xsi:nil="true"/>
<gn:nameStatus nilReason="missing" xsi:nil="true"/>
<gn:sourceOfName nilReason="missing" xsi:nil="true"/>
<gn:pronunciation nilReason="missing" xsi:nil="tru

In [73]:
codes_zone = []
departements = []
villes = [] 

for data_spots in all_xml_data_spots[1:]:
    try:
        code_zone = data_spots.find_all('base:localId')[0].text.split('-')[1].split('_')[0]
        departement = int(data_spots.find_all('ad:postCode')[0].text)
        ville = data_spots.find_all('gn:SpellingOfName')[0].find_all('gn:text')[0].text
    except:
        pass
    if not departement in departements:
        codes_zone.append(code_zone)
        departements.append(departement)
        villes.append(ville)
    
tbl_zones_prelev = pd.DataFrame({'code_zone':codes_zone,'departement':departements,'ville':villes})
tbl_zones_prelev.sort_values(['departement'],inplace=True)
tbl_zones_prelev

,code_zone,departement,ville
1,FR02001,13006,MARSEILLE
16,FR41001,20250,CORTE
10,FR26001,25000,BESANCON
6,FR08003,31300,TOULOUSE
7,FR09003,33692,MERIGNACÂ CEDEX
8,FR19002,35200,RENNES
9,FR23001,44307,NANTES CEDEX 3
11,FR34003,45590,SAINT-CYR-EN-VAL
4,FR06001,59800,LILLE
0,FR01001,67300,SCHILTIGHEIM


In [83]:
#Récupération du code associé à la zone "Paris"
a = 

# Récupération et retraitement des data xml sur la qualité de l'air
- https://files.data.gouv.fr/lcsqa/concentrations-de-polluants-atmospheriques-reglementes/old/2019/
- https://files.data.gouv.fr/lcsqa/concentrations-de-polluants-atmospheriques-reglementes/old/2020/
- https://files.data.gouv.fr/lcsqa/concentrations-de-polluants-atmospheriques-reglementes/old/2021/

#### Notes sur les différentes balises
-----------------------------------------------
```xml
<gml:TimePeriod gml:id="REP_TP_FR_2018-12-31-23_E2">
<gml:beginPosition>2018-09-19T00:00:00Z</gml:beginPosition>
<gml:endPosition>2018-10-10T00:00:00Z</gml:endPosition>
</gml:TimePeriod>
```
Période sur laquelle s'étend la donnée dans tout le fichier.

=================================================================================================

```xml
<aqd:content xlink:href="FR.LCSQA-INERIS.AQ/FR_OBS_et5tb64ner_FR070A_FR24040_7_1"/>
<aqd:content xlink:href="FR.LCSQA-INERIS.AQ/FR_OBS_86q294dv4i_FR070A_FR24039_5_2"/>
...
```
Points de prélèvement et polluants pour lesquelles les données sont fournies.

FR070A_FR24040_7 : [code reseau mesure]_ [code station]_ [n° polluant]

Numéros des polluants : http://dd.eionet.europa.eu/vocabulary/aq/pollutant/view?page=1#vocabularyConceptResults

=================================================================================================
```xml
<om:parameter>
<om:NamedValue>
<om:name xlink:href="http://dd.eionet.europa.eu/vocabulary/aq/processparameter/SamplingPoint"/>
<om:value xlink:href="FR.LCSQA-INERIS.AQ/SPO-FR24040_7"/> 
```
<code style="background:yellow;color:black">--- FR24040 : point de prélèvement</code>
```xml
</om:NamedValue>
</om:parameter>
<om:observedProperty xlink:href="http://dd.eionet.europa.eu/vocabulary/aq/pollutant/7"/> 
``` 
<code style="background:yellow;color:black">---> 7 : numéro de polluant</code>
```xml
<om:featureOfInterest xlink:href="FR.LCSQA-INERIS.AQ/SAM-FR24040_7"/>
<om:result>
<swe:DataArray>
<swe:elementCount>
<swe:Count>
<swe:value>24</swe:value>
</swe:Count>
</swe:elementCount>
``` 
<code style="background:yellow;color:black">---> 24 : nombre de données horaires disponibles sur la période de mesure</code>
```xml

<swe:elementType name="FixedObservations">
<swe:encoding>
<swe:TextEncoding blockSeparator="@@" decimalSeparator="." tokenSeparator=","/>
</swe:encoding>
<swe:values>2018-10-09T00:00:00+00:00,2018-10-09T01:00:00+00:00,2,1,61.0@@2018-10-09T01:00:00+00:00,2018-10-09T02:00:00+00:00,2,1,67.7@@2018-10-09T02:00:00+00:00,2018-10-09T03:00:00+00:00,2,1,65.0@@2018-10-09T03:00:00+00:00,2018-10-09T04:00:00+00:00,2,1,62.8@@2018-10-09T04:00:00+00:00,2018-10-09T05:00:00+00:00,2,1,66.5@@2018-10-09T05:00:00+00:00,2018-10-09T06:00:00+00:00,2,1,66.0@@</swe:values>
 ``` 
<code style="background:yellow;color:black">---> Groupe de valeurs séparées par "@@" </code>
    
<code style="background:yellow;color:black">2018-10-09T00:00:00+00:00,2018-10-09T01:00:00+00:00,2,1,61.0 = StartTime,EndTime,Verification (1), Validity (2), Value </code>

(1) voir tbl_verification


(2) voir tbl_validite

In [23]:
#Requête sur le code xml
r = requests.get(url_xml_test)
soup_data = BeautifulSoup(r.content,'xml')

Aperçu du résultat (non exhaustif) : 
    
```xml
...
<gml:featureMember>
...
</gml:featureMember>
<gml:featureMember>
...
<om:name xlink:href="http://dd.eionet.europa.eu/vocabulary/aq/processparameter/SamplingPoint"/>
<om:value xlink:href="FR.LCSQA-INERIS.AQ/SPO-FR24040_7"/>
...
<swe:Count>
<swe:value>24</swe:value>
</swe:Count>
...
<swe:encoding>
<swe:TextEncoding blockSeparator="@@" decimalSeparator="." tokenSeparator=","/>
</swe:encoding>
<swe:values>2018-10-09T00:00:00+00:00,2018-10-09T01:00:00+00:00,2,1,79.7@@2018-10-09T01:00:00+00:00,2018-10-09T02:00:00+00:00,2,1,85.7@@2018-10-09T02:00:00+00:00,2018-10-09T03:00:00+00:00,2,1,77.8@@2018-10-09T03:00:00+00:00,2018-10-09T04:00:00+00:00,2,1,80.8@@2018-10-09T04:00:00+00:00,2018-10-09T05:00:00+00:00,2,1,77.8@@2018-10-09T05:00:00+00:00,2018-10-09T06:00:00+00:00,2,1,73.3@@</swe:values>
...
</gml:featureMember>
```

In [24]:
#Séparation des différentes données
#Renvoie une liste dont l'élément 0 correspond aux infos globales relative au fichier. 
#Nous nous intéressons aux indices [1:-1].

all_xml_data = soup_data.find_all("gml:featureMember")

#all_xml_data

In [25]:
start_time = start = datetime.now()
#tbl_dataset = pd.DataFrame(columns=['Date','Heure début', 'Heure fin', 'Code point de prélèvement', 'Vérification','Validité', "Code polluant",'Valeur'])

dates = []
h_debut = []
h_fin = []
codes_prelev = []
verifs = []
valid = []
codes_pollu = []
valeurs = []

for data in all_xml_data[1:]:
    
    #Récupération du code du point de prélèvement et du polluant 
    ref = str(data.find_all("om:value")[1])
    lst = ref.split('-')[-1].split("\"")[0].split("_")
    
    code_spot_prelev = lst[0]
    code_polluant = lst[1]
    
    #Récupération des données
    values = data.find_all("swe:values")[0].text
    values_split = values.split('@@')[:-2]
    
    for elem in values_split:
        elem = elem.split(",")
        
        dates.append(elem[0][:9])
        h_debut.append(elem[0])
        h_fin.append(elem[1])
        codes_prelev.append(code_spot_prelev)
        verifs.append(elem[2])
        valid.append(elem[3])
        codes_pollu.append(code_polluant)
        valeurs.append(elem[4])
        
        """
        tbl_dataset = tbl_dataset.append({'Date': elem[0][:9],
                                          'Heure début' : elem[0],
                                          'Heure fin' : elem[1],
                                          'Code point de prélèvement': code_spot_prelev,
                                          'Vérification': elem[2],
                                          'Validité': elem[3],
                                          'Code polluant': code_polluant,
                                          'Valeur' : elem[4]}, ignore_index=True)
        """
        
        
        
tbl_dataset = pd.DataFrame({"date":dates,
                           "h début":h_debut,
                            "h fin":h_fin,
                            "codes_prelev":codes_prelev,
                            "verif":verifs,
                            "valid":valid,
                            "code polluant":codes_pollu,
                            "valeur":valeurs,
                           })
print("time", (datetime.now() - start))

tbl_dataset



time 0:00:01.795197


,date,h début,h fin,codes_prelev,verif,valid,code polluant,valeur
0,2018-10-0,2018-10-09T00:00:00+00:00,2018-10-09T01:00:00+00:00,FR24040,2,1,7,79.7
1,2018-10-0,2018-10-09T01:00:00+00:00,2018-10-09T02:00:00+00:00,FR24040,2,1,7,85.7
2,2018-10-0,2018-10-09T02:00:00+00:00,2018-10-09T03:00:00+00:00,FR24040,2,1,7,77.8
3,2018-10-0,2018-10-09T03:00:00+00:00,2018-10-09T04:00:00+00:00,FR24040,2,1,7,80.8
4,2018-10-0,2018-10-09T04:00:00+00:00,2018-10-09T05:00:00+00:00,FR24040,2,1,7,77.8
...,...,...,...,...,...,...,...,...
42270,2018-09-1,2018-09-19T18:00:00+00:00,2018-09-19T19:00:00+00:00,FR24007,2,1,7,70.5
42271,2018-09-1,2018-09-19T19:00:00+00:00,2018-09-19T20:00:00+00:00,FR24007,2,1,7,2.5
42272,2018-09-1,2018-09-19T20:00:00+00:00,2018-09-19T21:00:00+00:00,FR24007,2,1,7,9.5
42273,2018-09-1,2018-09-19T21:00:00+00:00,2018-09-19T22:00:00+00:00,FR24007,2,1,7,15.8


Récupération des data seulement